In [4]:
from numpy import mask_indices
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

embed=nn.Embedding(num_embeddings=10,embedding_dim=4)
#Embedding(10, 4)是一个10行4列的矩阵，随机生成，可以学习
#embed(x)是根据x的内容，返回对应的行，比如0行，1行，2，行，必须是0到9的整数
print(embed.weight.shape,embed.weight.dtype)
x=torch.tensor([[1,2,3],[4,5,6]])
y=embed(x)

print(y.shape,y.dtype)
class SigmoidBCELoss(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,inputs,target,mask=None):

        out=nn.functional.binary_cross_entropy_with_logits(inputs,target,weight=mask,reduction='none')
        return out.mean(dim=1)

loss=SigmoidBCELoss()
inputs=torch.randn(2,3)
target=torch.randn(2,3)
mask=torch.tensor([[1,1,0],[0,1,1]])
a=loss(inputs,target,mask)
print("a=",a)
import math
def sigmoid(x):
    a=1/(1+math.exp(-x))
    return -math.log(a)
print(sigmoid(0))


torch.Size([10, 4]) torch.float32
torch.Size([2, 3, 4]) torch.float32
a= tensor([0.4339, 0.0766])
0.6931471805599453


$w_i$表示第$i$个词，$v_i$表示词$w_i$作中心词的词向量，$u_i$表示$w_i$作上下文词的词向量。$q_{ij}=P(w_j|w_i)$表示$w_j$出现在$w_i$的上下文中的概率。
$$
\begin{aligned}
P(w_j|w_i) &= \frac{q_{ij}}{\sum_{k=1}^{n}q_{ik}} \\
&=\frac{\exp(u_j^T v_i)}{\sum_{k=1}^{n}\exp(u_k^T v_i)}
\end{aligned}
$$


GloVe模型,通过最小化词向量之间的余弦距离来学习词向量。

$$
\begin{aligned}
J &= \sum_{i=1}^{N}\sum_{j=1}^{N}h\left(x_{i j}\right)\left(u_{j}^{T} v_{i}+\boldsymbol{b}_{i}+\boldsymbol{c}_{j}-\log \left(x_{ij}\right)\right)^{2} \\


h\left(x\right) &= (x/c)^\alpha \\
\end{aligned}
$$

$x_{i j}$表示词$i$和词$j$的共现次数，$w_i$和$v_i$是词$i$作为中心词和上下文词的词向量，$\boldsymbol{b}_i$和$\boldsymbol{c}_j$是偏置项。

$$
exp(u_j^T v_i)\approx\alpha p_{ij},\qquad p_{ij}=x_{ij}/x_{i},
$$
两边取对数，$u_j^T v_i\approx\log \alpha + \log x_{ij}-\log x_{i}$

我们使用附加的偏置项来拟合$-\log \alpha+\log x_{i},$,

$$
u_j^T v_i+b_i+c_j\approx \log x_{ij}
$$


$fastText$通过"拆分词为子词，子词向量求和，表示整个词",

字节对编码(BPE,Byte Pair Encoding)


In [ ]:
import collections

class BPE:
    def __init__(self, num_merges=10):
        """
        初始化BPE对象.
        参数:
            num_merges: 需要合并的次数
        """
        self.num_merges = num_merges  # 设置合并次数
        self.bpe_codes = {}  # 保存合并规则, key为需要合并的符号对, value为合并顺序
    
    def get_stats(self, vocab):
        """
        计算当前词汇中所有符号对的频次.
        参数:
            vocab: dict, key为空格分隔的符号序列, value为词频
        返回:
            stats: dict, key为符号对(tuple), value为出现频次
        """
        stats = collections.defaultdict(int)  # 使用defaultdict初始化统计字典
        for word, freq in vocab.items():
            symbols = word.split()  # 将单词按空格分割成符号列表
            # 遍历所有相邻符号对，统计频次
            for i in range(len(symbols) - 1):
                stats[(symbols[i], symbols[i + 1])] += freq
        return stats

    def merge_vocab(self, pair, vocab):
        """
        在词汇中将符号对pair合并.
        参数:
            pair: tuple, 要合并的符号对
            vocab: dict, 当前词汇
        返回:
            new_vocab: dict, 更新后的词汇
        """
        new_vocab = {}
        bigram = ' '.join(pair)
        replacement = ''.join(pair)
        for word, freq in vocab.items():
            # 用合并后的符号替换原有的相邻符号
            new_word = word.replace(bigram, replacement)
            new_vocab[new_word] = freq
        return new_vocab

    def fit(self, data):
        """
        使用给定数据训练BPE模型.
        参数:
            data: list of str, 每个字符串代表一个单词或句子，这里按单词拆分
        """
        # 初始化词汇, 将每个单词拆分为字符，并用空格链接
        vocab = {}
        for word in data:
            tokens = ' '.join(list(word))
            vocab[tokens] = vocab.get(tokens, 0) + 1

        # 进行num_merges次的合并
        for i in range(self.num_merges):
            stats = self.get_stats(vocab)
            if not stats:
                break
            # 找到最频繁的符号对
            best_pair = max(stats, key=stats.get)
            self.bpe_codes[best_pair] = i
            vocab = self.merge_vocab(best_pair, vocab)
        self.vocab = vocab

    def encode(self, word):
        """
        根据训练好的BPE规则将单词编码为子词.
        参数:
            word: str, 输入的单词
        返回:
            tokens: list of str, BPE分词结果
        """
        tokens = list(word)
        # 按照合并顺序应用所有的BPE规则
        for pair, _ in sorted(self.bpe_codes.items(), key=lambda x: x[1]):
            bigram = list(pair)
            merged = ''.join(bigram)
            i = 0
            # 在tokens中查找连续匹配bigram的部分并进行合并
            while i < len(tokens) - 1:
                if tokens[i] == bigram[0] and tokens[i + 1] == bigram[1]:
                    tokens[i] = merged
                    del tokens[i + 1]
                else:
                    i += 1
        return tokens

# 定义训练与测试数据
if __name__ == "__main__":
    # 训练数据
    train_data = ["low", "lowest", "newer", "wider"]

    # 初始化BPE对象，设置合并次数
    bpe = BPE(num_merges=10)
    bpe.fit(train_data)

    print("训练得到的BPE合并规则:")
    for pair, order in bpe.bpe_codes.items():
        print(f"{pair} -> 合并顺序: {order}")

    # 测试编码
    test_words = ["lowest", "wider", "newer"]
    for word in test_words:
        encoded = bpe.encode(word)
        print(f"单词 '{word}' 的编码结果: {encoded}")

训练得到的BPE合并规则:
('l', 'o') -> 合并顺序: 0
('lo', 'w') -> 合并顺序: 1
('e', 'r') -> 合并顺序: 2
('low', 'e') -> 合并顺序: 3
('lowe', 's') -> 合并顺序: 4
('lowes', 't') -> 合并顺序: 5
('n', 'e') -> 合并顺序: 6
('ne', 'w') -> 合并顺序: 7
('new', 'er') -> 合并顺序: 8
('w', 'i') -> 合并顺序: 9
单词 'lowest' 的编码结果: ['lowest']
单词 'wider' 的编码结果: ['wi', 'd', 'er']
单词 'newer' 的编码结果: ['newer']


In [ ]:
import zipfile
import os

import urllib.request

url = "http://mattmahoney.net/dc/text8.zip"
filename = "text8.zip"
data_dir = "text8_data"

# Download dataset if not already downloaded
if not os.path.exists(filename):
    print("Downloading Text8 dataset...")
    urllib.request.urlretrieve(url, filename)
    print("Download complete.")

# Extract dataset to a designated directory if not already extracted
if not os.path.exists(data_dir):
    os.makedirs(data_dir, exist_ok=True)
    with zipfile.ZipFile(filename, "r") as zip_ref:
        zip_ref.extractall(data_dir)
    print("Dataset extracted to", data_dir)
else:
    print("Dataset already extracted in", data_dir)